# pymysql
- 파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
- Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
- https://github.com/PyMySQL/PyMySQL/
- https://pymysql.readthedocs.io/en/latest/

# 설치
- 조건
    - python version 3.6 이상
    - mysql version 5.6 이상
- 설치
    - `pip install pymysql`
    - `conda install -c conda-forge pymysql`

# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip", 
                                     port="port번호", 
                                     user="계정명", 
                                     password="비밀번호", 
                                     db="연결할데이터베이스이름", 
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다. 
    ```python
    cursor.close()
    connection.close()
    ```

# 예제

## 테이블 생성

In [1]:
CREATE_SQL = """
create table member(
id int auto_increment primary key,
name varchar(30) not null,
email varchar(100) not null unique,
tall decimal(5,2),
birthday date,
created_at datetime
)
"""

In [2]:
import pymysql

## 1. DB와 연결
conn = pymysql.connect(host="127.0.0.1", ## DB 서버 ip
                       port = 3306,      ## 연결할 DBMS port 번호
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'       ## 연결할 database 이름
                      )

print(type(conn))

## 2. connection 객체를 이용해 cursor(sql 작업을 관리)를 생성
cursor = conn.cursor()
print(type(cursor))

## 3. sql 문을 실행 - cursor execute()메소드 이용
cnt = cursor.execute(CREATE_SQL)
print("cnt:", cnt) #sql을 적용받은 데이터 수를 변환.

## database와 연결 닫기 ( 반드시 해야함. )
cursor.close()
conn.close()

<class 'pymysql.connections.Connection'>
<class 'pymysql.cursors.Cursor'>
cnt: 0


## DML
### insert

In [3]:
#sql문 -> mysql기준
insertSQL = "insert into member(name, email, tall, birthday, created_at) values('이순신', 'lee@a.com', '180.23', '1990-10-02',now())" #sql에서 ""을 줄바꿈 할때 \는 줄바꿈 포함시킴.

# 1.DB 연결.
with pymysql.connect(host = "127.0.0.1", port = 3306,
                    user = "hyunzuny", password = "qweqwe", db = "testdb") as conn:
    # 2.cursor 생성
    with conn.cursor() as cursor:
        # 3.sql문 실행
        cnt = cursor.execute(insertSQL)
        print('insert 개수 :', cnt)
        conn.commit()
        ### pymysql 은 수동 commit이 default임. 그래서
        
        
# 4.연결 닫기 -> cursor/DB -> with block 나오면서 자동으로 끊긴다.    #with open(~~~~) as f 파이썬에서 오픈문 비슷

insert 개수 : 1


### Parameterized Query
- Parameterized Query
    - SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    - query문을 쉽게 작성할 수 있는 장점이 있다.        

In [4]:
sql = "insert into member (name, email, tall, birthday, created_at)\
values(%s, %s, %s, %s, %s)"

In [5]:
import datetime
datetime.date(2000,1,2)
# datetime.datetime.now()

datetime.date(2000, 1, 2)

In [6]:
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(sql,("강감찬", "eqwd2a.com",180.3, datetime.date(2000,1,2), datetime.datetime.now()))
        
        conn.commit()


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용

In [7]:
from datetime import date, datetime

datas = [
    ['name1', 'abc1@abc.com', 165, date(2000,1,12), datetime.now()],
    ['name2', 'def1@abc.com', 175, date(1995,12,20), datetime.now()],
    ['name3', 'ghi1@abc.com', 185, date(1988, 7, 21), datetime.now()]
]

In [8]:
sql = "insert into member (name, email, tall, birthday, created_at)\
values(%s, %s, %s, %s, %s)"

with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt= 0
        for data  in datas:
            i = cursor.execute(sql,data)
            cnt += i
            conn.commit()

        
print(f'{cnt} : 행추가.')

3 : 행추가.


#### executemany() 사용
- insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.

In [9]:
datas = [
    ['이름1', 'abc2@a.com', 165, date(2000,1,12), datetime.now()],
    ['이름2', 'def2@a.com', 175, date(1995,12,20), datetime.now()],
    ['이름1', 'ghi2@a.com', 185, date(1988, 7, 21), datetime.now()]
]

In [10]:
sql = "insert into member (name, email, tall, birthday, created_at)\
values(%s, %s, %s, %s, %s)"

with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(sql, datas)
        print(cnt)
        conn.commit()

3


In [11]:
import datetime

### update/delete
- 코딩 절차는 insert 와 동일

In [18]:

update_sql = "update member set name=%s, email=%s, tall=%s, birthday=%s where id=%s"
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(update_sql,('새이름', 
                                         'hyunzuny@d.com',
                                         181.2,
                                         date(2000,11,11),
                                         2))
        conn.commit()

print(cnt)        

1


In [41]:
delete_sql = "delete from member where id = %s"
del_id = int(input("삭제할 ID:"))
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(delete_sql, del_id)
        conn.commit()
print(cnt)
        

삭제할 ID:4
1


In [42]:
result

{'id': 3,
 'name': 'name1',
 'email': 'abc1@abc.com',
 'tall': Decimal('165.00'),
 'birthday': datetime.date(2000, 1, 12),
 'created_at': datetime.datetime(2024, 1, 17, 14, 48, 10)}

## select (DQL - Data Query Language)
- 조회결과 조회
    - cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
- fetch메소드
    - **fetchall()**
        - 조회한 모든 행을을 반환
    - **fetchmany(size=개수)**
        - 지정한 size개수 만큼 반환
    - **fetchone()**
        - 조회결과 중 첫번째 행만 반환
        - 주로 pk 동등 조건으로 조회한 경우 사용

### fetchall()

In [21]:
import pymysql

In [22]:
from pprint import pprint

In [23]:
select_sql = "select * from member"
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(select_sql)
        # select 결과 조회 - fetchxxxx
        result = cursor.fetchall()
        

In [24]:
print(cnt) #조회 행수.

8


In [25]:
pprint(result)

((1,
  '이순신',
  'lee@a.com',
  Decimal('180.23'),
  datetime.date(1990, 10, 2),
  datetime.datetime(2024, 1, 17, 14, 48, 5)),
 (2,
  '새이름',
  'hyunzuny@d.com',
  Decimal('181.20'),
  datetime.date(2000, 11, 11),
  datetime.datetime(2024, 1, 17, 14, 48, 9)),
 (3,
  'name1',
  'abc1@abc.com',
  Decimal('165.00'),
  datetime.date(2000, 1, 12),
  datetime.datetime(2024, 1, 17, 14, 48, 10)),
 (4,
  'name2',
  'def1@abc.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 1, 17, 14, 48, 10)),
 (5,
  'name3',
  'ghi1@abc.com',
  Decimal('185.00'),
  datetime.date(1988, 7, 21),
  datetime.datetime(2024, 1, 17, 14, 48, 10)),
 (6,
  '이름1',
  'abc2@a.com',
  Decimal('165.00'),
  datetime.date(2000, 1, 12),
  datetime.datetime(2024, 1, 17, 14, 48, 12)),
 (7,
  '이름2',
  'def2@a.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 1, 17, 14, 48, 12)),
 (8,
  '이름1',
  'ghi2@a.com',
  Decimal('185.00'),
  datetime.date(1988, 7, 21),
  datet

### fetchone()

In [27]:
select_sql = "select * from member where id = %s"
id_num=int(input("조회할 ID :"))
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(select_sql, id_num)
        result = cursor.fetchone()

조회할 ID :1


In [28]:
print(cnt)

1


In [29]:
print(result)

(1, '이순신', 'lee@a.com', Decimal('180.23'), datetime.date(1990, 10, 2), datetime.datetime(2024, 1, 17, 14, 48, 5))


In [31]:
result[1], result[2]

('이순신', 'lee@a.com')

In [39]:

        
select_sql = "select * from member where id = %s"
id_num=int(input("조회할 ID :"))
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb',
                        cursorclass=pymysql.cursors.DictCursor) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(select_sql, id_num)
        result = cursor.fetchone()

조회할 ID :3


In [40]:
result

{'id': 3,
 'name': 'name1',
 'email': 'abc1@abc.com',
 'tall': Decimal('165.00'),
 'birthday': datetime.date(2000, 1, 12),
 'created_at': datetime.datetime(2024, 1, 17, 14, 48, 10)}

### fetchmany()

In [33]:
select_sql = "select * from member"
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(select_sql)
        # select 결과 조회 - fetchxxxx
        result1 = cursor.fetchmany(size = 2)
        result2 = cursor.fetchmany(size = 2)
        result3 = cursor.fetchmany(size = 2)
        result4 = cursor.fetchmany(size = 2)

In [30]:
print(cnt)

8


In [32]:
print(len(result))
result

2


((1,
  '이순신',
  'lee@a.com',
  Decimal('180.23'),
  datetime.date(1990, 10, 2),
  datetime.datetime(2024, 1, 17, 12, 28, 56)),
 (2,
  '이름1',
  'abc2@a.com',
  Decimal('165.00'),
  datetime.date(2000, 1, 12),
  datetime.datetime(2024, 1, 17, 12, 49, 6)))

In [35]:
pprint(result2)

((3,
  '이름2',
  'def2@a.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 1, 17, 12, 49, 6)),
 (4,
  '이름1',
  'ghi2@a.com',
  Decimal('185.00'),
  datetime.date(1988, 7, 21),
  datetime.datetime(2024, 1, 17, 12, 49, 6)))


### cursor 는 iterable 타입
- for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.

In [69]:
select_sql = "select * from member where name like %s"
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(select_sql,'n%' )
        result = cursor.fetchall()
        sum_tall=0
        for data in cursor:
            sum_tall+=float(data[3])

        
        

print(cnt)
pprint(result)

2
((3,
  'name1',
  'abc1@abc.com',
  Decimal('165.00'),
  datetime.date(2000, 1, 12),
  datetime.datetime(2024, 1, 17, 14, 48, 10)),
 (5,
  'name3',
  'ghi1@abc.com',
  Decimal('185.00'),
  datetime.date(1988, 7, 21),
  datetime.datetime(2024, 1, 17, 14, 48, 10)))


In [70]:
sum_tall

0

# 함수 구현

보통 테이블과 관련해서 CRUD를 처리하는 함수 또는 클래스를 정의해서 사용한다.  
다음은 Member 테이블의 CRUD를 처리하는 함수를 작성한다.

1. name, email, tall, birthday를 매개변수로 받아서 insert하는 함수. (id는 자동증가, created_at은 실행시점의 일시가 insert되도록 한다.)
2. id, name, email, tall, birthday를 매개변수로 받아서 id의 member의 나머지 정보를 update하는 함수. (created_at은 update하지 않는다.)
3. id를 매개변수로 받아서 그 member 를 삭제하는 함수.
4. id를 매개변수로 받아서 그 id의 회원 정보를 조회하여 반환하는 함수.
5. 전체 회원정보를 조회하는 함수


- insert, update, delete 는 적용된 행의 개수를 반환한다.
- select 처리 함수는 조회결과를 반환한다.


In [93]:
%%writefile member_dao.py

def insert_member(corsor, name, email, tall, birthday):
    
    sql = 'insert into member (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())'
    return cursor.execute(sql, (name, email, tall, birthday))
     
def update_member(cursor, id, name, email, tail, birthday):
    sql = "update member set name = %s, email = %s, tall =%s, birthday = %s where id =%s"
    return cursor.execute(sql, (name, email, tail, birthday, id))

def delete_memberb_by_id(sursor, id):
    sql = "delete from member where id=%s"
    return cursor.execute(sql, id)

def select_members(cursor):
    sql = "select * from member"
    cursor.execute(sql)
    return cursor.fetchall()

def select_member_by_id(cursor, id):
    sql = "select*from member where id =%s"
    cursor.execute(sql,id)
    return cursor.fetchone()

Writing member_dao.py


In [94]:
from member_dao import *

In [95]:
with pymysql.connect(host="127.0.0.1", 
                       port = 3306,     
                       user = "hyunzuny",
                       password = "qweqwe",
                       db='testdb'      
                      ) as conn:
    with conn.cursor() as cursor:
        #id = 5 를 조회 후 이름을 변경
        result = list(select_member_by_id(cursor, 2))
        print(result)
        result[1] = "새로운 이름입니다."
        if len(result) ==0:
            conn.rollback()
            raise Exception() # 예외발생
        update_member(cursor, result[0], result[1], result[2], result[3], result[4])
        
        
        
        

[2, '새이름', 'hyunzuny@d.com', Decimal('181.20'), datetime.date(2000, 11, 11), datetime.datetime(2024, 1, 17, 14, 48, 9)]
